# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

In [2]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


┌ Warning: Package LinearAlgebraicRepresentation does not have BenchmarkTools in its dependencies:
│ - If you have LinearAlgebraicRepresentation checked out for development and have
│   added BenchmarkTools as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearAlgebraicRepresentation
└ Loading BenchmarkTools into LinearAlgebraicRepresentation from project dependency, future warnings for LinearAlgebraicRepresentation are suppressed.


# FUNZIONI AGGIUNTIVE

### RemoveIntersection

rimuove le intersezioni contenute in **covers** che i boundingbox hanno con se stessi



In [3]:
function removeIntersection(covers::Array{Array{Int64,1},1})
    @threads for k=1:length(covers)
        covers[k] = setdiff(covers[k],[k])	#toglie le intersezioni con se stesso 
    end
end

removeIntersection (generic function with 1 method)

### CreateIntervalTree

funzione che crea un albero di supporto per la funzione principale **boxcovering**; nel particolare dato un dizionario ordinato crea un intervalTrees cioè una struttura dati che contiene intervalli e che consente di trovare in modo efficiente tutti gli intervalli che si sovrappongono a un determinato intervallo o punto.

In [4]:
function createIntervalTree(boxdict::AbstractDict{Array{Float64,1},Array{Int64,1}})
    tree = IntervalTrees.IntervalMap{Float64,Array}()
    for (key, boxset) in boxdict
        tree[tuple(key...)] = boxset
    end
    return tree
end

createIntervalTree (generic function with 1 method)

## addIntersection

addIntersection aggiunge in 'covers' in i-esima posizione tutti i bounding box che intersecano l'i-esimo bounding box


In [5]:
function addIntersection(covers::Array{Array{Int64,1},1}, i::Int64, iterator)
    splice!(covers[i],1)		#splice serve a togliere gli zeri iniziali all'interno di covers
    @threads for x in collect(iterator)
        append!(covers[i],x.value)
    end
end

addIntersection (generic function with 1 method)

## TEST

In [ ]:
@testset "removeIntersection test" begin
    covers = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]
    removeIntersection(covers)
    @test typeof(covers) == Array{Array{Int64,1},1}
    @test covers[1] == [4, 3, 5, 2]
    @test covers[2] == [1, 3, 5]
    @test covers[3] == [4, 1, 5, 2]
    @test covers[4] == [1, 3, 5]
    @test covers[5] == [4, 1, 3, 2]
end

In [ ]:
@testset "createIntervalTree test" begin
    dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])
    t = createIntervalTree(dict)
    @test typeof(t) == IntervalTrees.IntervalBTree{Float64,IntervalValue{Float64,Array},64}
    @test t.root.entries[1].first == 0.0
    @test t.root.entries[1].last == 0.0
    @test t.root.entries[1].value == [4]

    @test t.root.entries[2].first == 0.0
    @test t.root.entries[2].last == 1.0
    @test t.root.entries[2].value == [1,3]

    @test t.root.entries[3].first == 0.0
    @test t.root.entries[3].last == 2.0
    @test t.root.entries[3].value == [5]

    @test t.root.entries[4].first == 1.0
    @test t.root.entries[4].last == 1.0
    @test t.root.entries[4].value == [2]
end


In [ ]:
@testset "addIntersection test" begin
    bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];
    dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])
    t = createIntervalTree(dict)
    c = boxcovering(bb,1,t)
    @test c == [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]];
end

![test di spaceindex](https://github.com/MarcoCap13/LAR-SPLITTING-2D-5.b-/blob/main/docs/test/spaceindex_test.png?raw=true)